# Hybrid VAE Recommendation System - Complete Pipeline Demo

This notebook demonstrates the complete pipeline for building and using a Hybrid Variational Autoencoder recommendation system that combines collaborative filtering with content-based filtering using SBERT embeddings.

## Overview

The system consists of:
1. **Data Preprocessing**: Loading and cleaning Amazon Reviews data
2. **Dataset Building**: Creating user-item matrices and train/val/test splits
3. **Embeddings Computation**: Using SBERT to encode item text
4. **Model Training**: Training the Hybrid VAE
5. **Evaluation**: Computing Recall@K and NDCG@K metrics
6. **API Usage**: Serving recommendations via FastAPI

## Setup and Imports

In [ ]:
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import torch
import json
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

# Import our custom modules
from preprocessing import preprocess_amazon_reviews
from build_dataset import build_recommendation_dataset
from compute_embeddings import compute_item_embeddings
from train_vae import train_hybrid_vae
from evaluate import evaluate_recommendation_model
from utils import *

# Set up logging
setup_logging("INFO")

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

print("Setup complete!")

## 1. Data Preprocessing

First, we'll load and preprocess the Amazon Reviews dataset. This step:
- Loads JSONL data
- Cleans missing values
- Creates `item_text` field by combining title and review text
- Converts ratings to binary (rating >= 4 → 1)
- Filters users and items with minimum interactions

In [ ]:
# Define paths
data_dir = Path('../data')
raw_data_path = data_dir / 'reviews.jsonl'  # Your raw Amazon Reviews data
processed_data_path = data_dir / 'cleaned_reviews.jsonl'

# Check if raw data exists
if not raw_data_path.exists():
    print(f"⚠️  Raw data not found at {raw_data_path}")
    print("Please place your Amazon Reviews JSONL file there to continue.")
    print("For demo purposes, we'll create a small sample dataset.")
    
    # Create a small sample dataset for demonstration
    sample_data = []
    for i in range(1000):
        sample_data.append({
            'user_id': f'user_{i % 100}',
            'asin': f'item_{i % 50}',
            'rating': np.random.choice([3, 4, 5], p=[0.2, 0.4, 0.4]),
            'title': f'Product {i % 50} Title',
            'text': f'This is a review for product {i % 50}. Great quality and value.',
            'timestamp': 1600000000 + i * 1000
        })
    
    # Save sample data
    with open(raw_data_path, 'w') as f:
        for item in sample_data:
            f.write(json.dumps(item) + '\n')
    
    print(f"✅ Created sample dataset with {len(sample_data)} interactions")

print(f"Raw data available at: {raw_data_path}")

In [ ]:
# Run preprocessing
print("🔄 Starting data preprocessing...")

preprocess_amazon_reviews(
    input_path=str(raw_data_path),
    output_path=str(processed_data_path),
    min_user_interactions=3,  # Lower threshold for demo
    min_item_interactions=3,
    rating_threshold=4.0
)

print("✅ Preprocessing complete!")

In [ ]:
# Load and examine the preprocessed data
df = pd.read_csv(processed_data_path.with_suffix('.csv'))

print(f"Preprocessed dataset shape: {df.shape}")
print(f"Unique users: {df['user_id'].nunique()}")
print(f"Unique items: {df['asin'].nunique()}")
print(f"Positive interactions: {df['binary_rating'].sum()}")

# Display sample data
print("\nSample data:")
display(df.head())

# Compute and display statistics
stats = compute_dataset_statistics(df)
print("\nDataset Statistics:")
for key, value in stats.items():
    if isinstance(value, dict):
        print(f"{key}:")
        for subkey, subvalue in value.items():
            print(f"  {subkey}: {subvalue:.2f}")
    else:
        print(f"{key}: {value:.4f}" if isinstance(value, float) else f"{key}: {value}")

## 2. Dataset Building

Next, we'll create the user-item interaction matrix and train/validation/test splits using leave-one-out methodology.

In [ ]:
# Build the recommendation dataset
dataset_dir = data_dir / 'processed_dataset'

print("🔄 Building recommendation dataset...")

build_recommendation_dataset(
    input_path=str(processed_data_path.with_suffix('.csv')),
    output_dir=str(dataset_dir),
    add_negatives=True,
    n_negatives_per_positive=2  # Lower for demo
)

print("✅ Dataset building complete!")

In [ ]:
# Examine the created dataset
train_df = pd.read_csv(dataset_dir / 'train.csv')
val_df = pd.read_csv(dataset_dir / 'val.csv')
test_df = pd.read_csv(dataset_dir / 'test.csv')

# Load dataset statistics
with open(dataset_dir / 'dataset_stats.pkl', 'rb') as f:
    dataset_stats = pickle.load(f)

print("Dataset Splits:")
print(f"Train: {len(train_df):,} interactions")
print(f"Val: {len(val_df):,} interactions")
print(f"Test: {len(test_df):,} interactions")

print("\nDataset Statistics:")
for key, value in dataset_stats.items():
    print(f"{key}: {value}")

# Validate data consistency
consistency_results = validate_data_consistency(str(dataset_dir))
print("\nData Consistency Check:")
for check, passed in consistency_results.items():
    status = "✅" if passed else "❌"
    print(f"{status} {check}: {passed}")

## 3. Compute Item Embeddings

We'll use SBERT to create dense vector representations of items based on their text (title + review text).

In [ ]:
# Compute item embeddings using SBERT
embeddings_dir = Path('../embeddings')
embeddings_path = embeddings_dir / 'item_embeddings.npy'

print("🔄 Computing item embeddings with SBERT...")
print("This may take a few minutes depending on the dataset size.")

compute_item_embeddings(
    input_path=str(processed_data_path.with_suffix('.csv')),
    output_path=str(embeddings_path),
    model_name="all-MiniLM-L6-v2",
    batch_size=16,  # Smaller batch size for demo
    device='cpu'  # Use CPU for demo
)

print("✅ Embeddings computation complete!")

In [ ]:
# Examine the computed embeddings
from compute_embeddings import load_embeddings

embeddings, item_to_idx, idx_to_item = load_embeddings(
    str(embeddings_path),
    str(embeddings_path.with_name('item_embeddings_mappings.pkl'))
)

print(f"Embeddings shape: {embeddings.shape}")
print(f"Number of items: {len(item_to_idx)}")
print(f"Embedding dimension: {embeddings.shape[1]}")

# Show some statistics about embeddings
print(f"\nEmbedding Statistics:")
print(f"Mean: {embeddings.mean():.4f}")
print(f"Std: {embeddings.std():.4f}")
print(f"Min: {embeddings.min():.4f}")
print(f"Max: {embeddings.max():.4f}")

# Plot embedding distribution
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
plt.hist(embeddings.flatten(), bins=50, alpha=0.7)
plt.title('Distribution of Embedding Values')
plt.xlabel('Value')
plt.ylabel('Frequency')

plt.subplot(1, 2, 2)
norms = np.linalg.norm(embeddings, axis=1)
plt.hist(norms, bins=30, alpha=0.7)
plt.title('Distribution of Embedding Norms')
plt.xlabel('L2 Norm')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()

## 4. Model Training

Now we'll train the Hybrid VAE model that combines collaborative filtering with the item embeddings.

In [ ]:
# Train the Hybrid VAE model
models_dir = Path('../models')
experiment_name = 'hybrid_vae_demo'
model_output_dir = models_dir / experiment_name

print("🔄 Training Hybrid VAE model...")
print("This will take several minutes. Check the progress above.")

train_hybrid_vae(
    data_dir=str(dataset_dir),
    embeddings_path=str(embeddings_path),
    output_dir=str(model_output_dir),
    latent_dim=64,  # Smaller for demo
    hidden_dims=[128, 64],  # Smaller for demo
    batch_size=64,  # Smaller for demo
    epochs=20,  # Fewer epochs for demo
    learning_rate=0.001,
    beta=0.2,
    dropout=0.5,
    patience=5,
    device='cpu'  # Use CPU for demo
)

print("✅ Training complete!")

In [ ]:
# Analyze training history
with open(model_output_dir / 'training_history.json', 'r') as f:
    history = json.load(f)

print("Training Summary:")
print(f"Final training loss: {history['train_losses'][-1]:.4f}")
if history['val_losses']:
    print(f"Final validation loss: {history['val_losses'][-1]:.4f}")
print(f"Best validation loss: {min(history['val_losses']) if history['val_losses'] else 'N/A'}")

# Plot training curves
plot_training_history(history, save_path=str(model_output_dir / 'training_plot.png'))

## 5. Model Evaluation

Let's evaluate the trained model using Recall@K and NDCG@K metrics on the test set.

In [ ]:
# Evaluate the model
best_model_path = model_output_dir / 'best_model.pth'

print("🔄 Evaluating model performance...")

results = evaluate_recommendation_model(
    model_path=str(best_model_path),
    data_dir=str(dataset_dir),
    embeddings_path=str(embeddings_path),
    k_values=[5, 10, 20],
    device='cpu'
)

print("✅ Evaluation complete!")

In [ ]:
# Display evaluation results
print("\n" + "="*50)
print("EVALUATION RESULTS")
print("="*50)

for k in [5, 10, 20]:
    if k in results:
        metrics = results[k]
        print(f"\n@{k}:")
        print(f"  Recall:    {metrics['recall']:.4f}")
        print(f"  NDCG:      {metrics['ndcg']:.4f}")
        print(f"  Hit Ratio: {metrics['hit_ratio']:.4f}")

# Save results
with open(model_output_dir / 'evaluation_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print(f"\n📊 Results saved to: {model_output_dir / 'evaluation_results.json'}")

## 6. Generate Sample Recommendations

Let's generate some sample recommendations to see the model in action.

In [ ]:
# Load the trained model for inference
from evaluate import load_model_from_checkpoint, RecommendationEvaluator

# Load data and model
interaction_matrix, train_df, val_df, mappings = load_training_data(str(dataset_dir))
user_to_idx = mappings['user_to_idx']
item_to_idx = mappings['item_to_idx']
idx_to_user = mappings['idx_to_user']
idx_to_item = mappings['idx_to_item']

model = load_model_from_checkpoint(str(best_model_path), embeddings, torch.device('cpu'))

# Create evaluator for generating recommendations
evaluator = RecommendationEvaluator(
    model=model,
    interaction_matrix=interaction_matrix,
    user_to_idx=user_to_idx,
    item_to_idx=item_to_idx,
    device=torch.device('cpu')
)

print("Model loaded and ready for recommendations!")

In [ ]:
# Generate recommendations for sample users
sample_users = list(user_to_idx.keys())[:5]  # First 5 users

print("Sample Recommendations:")
print("="*80)

for user_id in sample_users:
    if user_id in user_to_idx:
        user_idx = user_to_idx[user_id]
        
        # Get user's interaction history
        user_items = interaction_matrix[user_idx].nonzero()[1]
        interacted_items = [idx_to_item[item_idx] for item_idx in user_items[:5]]
        
        # Generate recommendations
        recommended_indices, scores = evaluator.get_user_recommendations(user_idx, top_k=10)
        recommended_items = [(idx_to_item[idx], scores[i]) 
                           for i, idx in enumerate(recommended_indices[:5])]
        
        print(f"\nUser: {user_id}")
        print(f"Items interacted with: {interacted_items}")
        print("Top 5 Recommendations:")
        for i, (item_id, score) in enumerate(recommended_items, 1):
            print(f"  {i}. {item_id} (score: {score:.3f})")

## 7. API Usage Demo

Finally, let's demonstrate how to use the FastAPI server for serving recommendations.

In [ ]:
# Note: In practice, you would run the API server in a separate process
# Here we'll simulate the API functionality

from api import create_app
import requests
import json

print("🔄 Setting up API demo...")

# Create the FastAPI app (this loads the model)
try:
    app = create_app(
        model_path=str(best_model_path),
        data_dir=str(dataset_dir),
        embeddings_path=str(embeddings_path),
        device_name='cpu'
    )
    print("✅ API app created successfully!")
    print("\nTo run the API server, use:")
    print(f"python ../src/api.py --model {best_model_path} --data {dataset_dir} --embeddings {embeddings_path}")
    print("\nThen you can make requests to http://localhost:8000")
except Exception as e:
    print(f"❌ Error creating API app: {e}")
    print("This is expected in notebook environment. Run the API server separately.")

In [ ]:
# Example API requests (you would use these when the server is running)

print("Example API Usage:")
print("="*50)

# Health check
print("1. Health Check:")
print("GET /health")
print("Response: {\"status\": \"healthy\", \"model_loaded\": true, ...}")

# Get recommendations
print("\n2. Get Recommendations:")
print("POST /recommend")
example_request = {
    "user_id": list(user_to_idx.keys())[0],
    "top_k": 10,
    "exclude_seen": True
}
print(f"Request Body: {json.dumps(example_request, indent=2)}")
print("Response: {\"user_id\": \"...\", \"recommendations\": [...], ...}")

# User profile
print("\n3. Get User Profile:")
print(f"GET /users/{list(user_to_idx.keys())[0]}/profile")
print("Response: {\"user_id\": \"...\", \"total_interactions\": 42, ...}")

# Batch recommendations
print("\n4. Batch Recommendations:")
print("POST /recommend/batch")
batch_request = {
    "user_ids": list(user_to_idx.keys())[:3],
    "top_k": 5
}
print(f"Request Body: {json.dumps(batch_request, indent=2)}")

print("\n💡 Start the API server and use curl or any HTTP client to test these endpoints!")

## Summary

In this notebook, we've demonstrated the complete pipeline for building a Hybrid VAE recommendation system:

1. **✅ Data Preprocessing**: Cleaned and prepared Amazon Reviews data
2. **✅ Dataset Building**: Created user-item matrices and proper train/val/test splits
3. **✅ Embeddings**: Generated SBERT embeddings for item text
4. **✅ Model Training**: Trained the Hybrid VAE combining collaborative and content-based filtering
5. **✅ Evaluation**: Measured performance using Recall@K and NDCG@K
6. **✅ Recommendations**: Generated sample recommendations
7. **✅ API Setup**: Demonstrated how to serve the model via FastAPI

### Next Steps:

1. **Scale Up**: Use larger datasets and adjust hyperparameters
2. **Hyperparameter Tuning**: Experiment with different latent dimensions, learning rates, etc.
3. **Advanced Features**: Add user features, temporal dynamics, or cold-start handling
4. **Production**: Deploy the API with proper monitoring and scaling
5. **A/B Testing**: Compare with other recommendation algorithms

### Key Benefits of This Approach:

- **Hybrid**: Combines collaborative filtering strength with content understanding
- **Scalable**: VAE approach handles sparse data well
- **Interpretable**: Item embeddings provide insight into content relationships
- **Production-Ready**: Complete API for serving recommendations
- **Extensible**: Modular design allows easy improvements and experiments